# ETL IV Pair 24/08


In [ ]:
import pandas as pd
import mysql.connector

## ETL Carga I

1. Cread la BBDD con el nombre de energía. La BBDD tendrá la siguiente forma:
2. Cread las tablas de la BBDD:
- Tabla fechas
- Tabla nacional_renovable_no_renovable
- Tabla comunidades_renovable_no_renovable
- Tabla comunidades

Al final nuestra BBDD tendrá el siguiente aspecto:
    
3. BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [ ]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab", 
      auth_plugin = 'mysql_native_password') 
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
mydb = crear_bbdd("energia")

In [ ]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
 
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}",  
                                     auth_plugin = 'mysql_native_password')

    mycursor = cnx.cursor()
    
   
    try: 
        mycursor.execute(query)
        cnx.commit() 
   
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
tabla_fechas = '''
CREATE TABLE IF NOT EXISTS `energia`.`fechas` (
  `idfechas` INT NOT NULL AUTO_INCREMENT,
  `fecha` DATE NULL,
  PRIMARY KEY (`idfechas`))
ENGINE = InnoDB;
'''

In [ ]:
tabla_nacional_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_fechas_idx` (`fechas_idfechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_fechas`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;
'''

In [ ]:
tabla_comunidades = '''
CREATE TABLE IF NOT EXISTS `energia`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidades` VARCHAR(45) NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
'''

In [ ]:
tabla_comunidades_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_idfechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `energia`.`comunidades` (`idcomunidades`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;
'''

In [ ]:
tablas = [tabla_fechas, tabla_nacional_renovable_no_renovable, tabla_comunidades, tabla_comunidades_renovable_no_renovable]

for tabla in tablas:
    crear_insertar_tabla("energia","AlumnaAdalab", tabla)

## Bonus

### Validación: No hay datos insertados

In [ ]:
def check_comunidades(contraseña, nombre_bbdd):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{contraseña}",
                                      host='127.0.0.1',
                                      database=f"{nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_ciudad = f"""
                SELECT DISTINCT comunidades FROM comunidades
                """
        mycursor.execute(query_existe_ciudad)
        comunidades = mycursor.fetchall()
        return comunidades
    
    # método para sacar el id de una ciudad en concreto 
def sacar_id_comunidad(comunidad, contraseña, nombre_bbdd):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidades FROM comunidades WHERE comunidades = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. "
     
    
def sacar_id_fecha(fecha, contraseña, nombre_bbdd):
        mydb = mysql.connector.connect(user='root', password=f'{contraseña}',
                                          host='127.0.0.1', database=f"{nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idfechas FROM fechas WHERE fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [ ]:
id_comunidad = sacar_id_comunidad("Madrid","AlumnaAdalab","energia")
id_comunidad

In [ ]:
id_fecha = sacar_id_fecha(2011-11-10, "AlumnaAdalab","energia")
id_fecha

### Inserción de datos en la tabla 'fechas'

Como puede haber fechas que no existan en el dataframe 'df_energia_ccaa' o en el dataframe 'df_energia_nacional', se van a seguir los siguientes pasos:

1. Unir ambos dataframes por la columna 'fecha'.
2. Como posiblemente, haya fechas duplicadas, eliminamos los posibles duplicados.
3. Insertamos en la tabla 'fechas' los registros únicos de la columna 'fecha' de nuestro nuevo dataframe.

In [ ]:
df_energia_ccaa = pd.read_pickle("data/pairETL3_energia_ccaa.pkl")
df_energia_ccaa.head(2)

In [ ]:
df_energia_nacional = pd.read_pickle("data/pairETL3_energia_nacional.pkl")
df_energia_nacional.head(2)

In [ ]:
df_fechas = pd.concat([df_energia_ccaa['fecha'], df_energia_nacional['fecha']], axis = 0, join = "inner", ignore_index = True)
df_fechas = pd.DataFrame(df_fechas)
df_fechas

In [ ]:
df_fechas.duplicated().sum()

In [ ]:
df_fechas = df_fechas.drop_duplicates()
df_fechas

In [ ]:
df_fechas.duplicated().sum()

In [ ]:
for indice, fila in df_fechas.iterrows(): 
    query_fechas = f"""
            INSERT INTO fechas (fecha) 
            VALUES ("{fila['fecha']}");
            """
    crear_insertar_tabla(query_fechas, "AlumnaAdalab","energia")

### Inserción de datos en la tabla 'comunidades'

Los dataframe que podemos utilizar son 'df_energia_ccaa' y  'df_poblacion_ccaa'. En este caso, vamos a utilizar el dataframe 'df_energia_ccaa' porque así tendremos los mismos nombres que vamos a tener que enlazar cuando hagamos la inserción de los datos de energia de las CCAA.

1. Vamos a crear un nuevo dataframe con los registros de la columna 'comunidad'
2. Eliminamos los posibles duplicados del nuevo dataframe.
3. Insertamos en la tabla 'comunidades' los registros únicos de la columna 'comunidad' de nuestro nuevo dataframe.

In [ ]:
df_comunidades = df_energia_ccaa[['comunidad']]
df_comunidades.sample()

In [ ]:
df_comunidades.duplicated().sum()

In [ ]:
df_comunidades = df_comunidades.drop_duplicates()

In [ ]:
df_comunidades.duplicated().sum()

In [ ]:
df_comunidades

In [ ]:
for indice, fila in df_comunidades.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidad = f"""
                INSERT INTO comunidades (comunidades) 
                VALUES ("{fila['comunidad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    comunidades = check_comunidades("AlumnaAdalab","energia")
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(comunidades) == 0 or fila['comunidad'] not in comunidades[0]: 
        crear_insertar_tabla(query_comunidad, "AlumnaAdalab","energia")

    else:
        print(f"{fila['comunidades']} ya esta en nuestra BBDD")

### Inserción de los datos en la tabla 'comunidades_renovable_no_renovable'

El dataframe que vamos a utilizar es 'df_energia_ccaa'.

1. Insertamos en la tabla 'comunidades_renovable_no_renovable' los registros del dataframe, buscando los ids correspondientes a la columna 'fecha' y 'comunidad' para poder enlazar correctamente la información de la BBDD.

In [ ]:
df_energia_ccaa.head(2)

In [ ]:
for indice, fila in df_energia_ccaa.iterrows():
    
    id_comunidad = sacar_id_comunidad(fila['comunidad'], "AlumnaAdalab","energia")
    id_fecha = sacar_id_fecha(fila["fecha"], "AlumnaAdalab","energia")
        
    query_valores_ccaa = f"""
                INSERT INTO comunidades_renovable_no_renovable (porcentaje, tipo_energia, valor, fechas_idfechas, comunidades_idcomunidades) 
                VALUES ({fila['percentage']}, "{fila['tipo_energia']}", {fila['value']}, {id_fecha}, {id_comunidad});
                """
    crear_insertar_tabla(query_valores_ccaa, "AlumnaAdalab","energia")
    

### Inserción de los datos en la tabla 'nacional_renovable_no_renovable'

El dataframe que vamos a utilizar es 'df_energia_nacional'.

1. Insertamos en la tabla 'nacional_renovable_no_renovable' los registros del dataframe, buscando los ids correspondientes a la columna 'fecha' para poder enlazar correctamente la información de la BBDD.

In [ ]:
for indice, fila in df_energia_nacional.iterrows():
    
    id_fecha = sacar_id_fecha(fila["fecha"], "AlumnaAdalab","energia")
    
    query_valores_nacional = f"""
                INSERT INTO nacional_renovable_no_renovable (porcentaje, tipo_energia, valor, fechas_idfechas)
                VALUES ({fila['percentage']}, "{fila['tipo_energia']}", {fila['value']}, {id_fecha});
                """
    crear_insertar_tabla(query_valores_nacional, "AlumnaAdalab","energia")
    